In [6]:
%pip install neo4j
from graphclass import BioAssayGraphDatabase
import pandas as pd
import requests
import json




[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
def get_cids_from_sid(sid:int): 
    # Replace with the actual Substance ID

    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sid/{sid}/cids/JSON"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        cids = data.get('InformationList', {}).get('Information', [{}])[0].get('CID', [])
        print(f"Substance ID: {sid}, Compound IDs: {cids}")
        return [True , cids]
    else:
        print(f"Failed to retrieve compound IDs for substance ID {sid}: {response.status_code}")
        return [False, -999]
def get_TypeSidOfAcitivity_FromAid(aid:int):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{aid}/JSON"

    # Make a GET request to the URL
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        assay_data = response.json()
        #print(json.dumps(assay_data, indent=4))
        # Iterate over the results and print the Substance ID and Activity Outcome
        #get all key of assay_data
        #print(assay_data['PC_AssaySubmit'])
        #print("______________________________ch____________________________")
        #print(json.dumps(assay_data['PC_AssaySubmit']['data'] , indent=4))
        return [True,assay_data['PC_AssaySubmit']['data']]
            
    else:
        print(f"Failed to retrieve assay results: {response.status_code}") 
        return [False, -999]


In [8]:
print(get_cids_from_sid(426232))

Substance ID: 426232, Compound IDs: [284243]
[True, [284243]]


In [9]:
# Usage Example:
uri = "bolt://localhost:7687"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your Neo4j username
password = "root12345"  # Replace with your Neo4j password

db = BioAssayGraphDatabase(uri, username, password)


In [10]:
# Replace 'your_file.csv' with the path to your CSV file
file_path_compound = 'data/compound.csv'
file_path_bioassay_bio = 'data/bioassay_assay.csv' 
 
# Read the CSV into a pandas data frame (df)
df_compound = pd.read_csv(file_path_compound, sep=',')
df_bioassay_bio = pd.read_csv(file_path_bioassay_bio, sep=',')


/var/folders/tw/ksffzqq93t56xq1d5v8_ldyc0000gn/T/ipykernel_1921/798445942.py:6: DtypeWarning: Columns (32,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_compound = pd.read_csv(file_path_compound, sep=',')
/var/folders/tw/ksffzqq93t56xq1d5v8_ldyc0000gn/T/ipykernel_1921/798445942.py:7: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bioassay_bio = pd.read_csv(file_path_bioassay_bio, sep=',')


In [11]:
#show head of df_compound 
pd.set_option('display.max_columns', None)
df_compound.head() 
# how to for in every row 


,cid,cmpdname,cmpdsynonym,mw,mf,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,inchi,isosmiles,canonicalsmiles,inchikey,iupacname,exactmass,monoisotopicmass,charge,covalentunitcnt,isotopeatomcnt,totalatomstereocnt,definedatomstereocnt,undefinedatomstereocnt,totalbondstereocnt,definedbondstereocnt,undefinedbondstereocnt,pclidcnt,gpidcnt,gpfamilycnt,neighbortype,meshheadings,annothits,annothitcnt,aids,cidcdate,sidsrcname,depcatg,annotation
0,243,Benzoic Acid,benzoic acid|65-85-0|Dracylic acid|benzenecarb...,122.120,C7H6O2,37.3,104.0,1.9,9,1,2,1,"InChI=1S/C7H6O2/c8-7(9)6-4-2-1-3-5-6/h1-5H,(H,...",C1=CC=C(C=C1)C(=O)O,C1=CC=C(C=C1)C(=O)O,WPYMKLBDIGXBTP-UHFFFAOYSA-N,benzoic acid,122.0370,122.0370,0,1,0,0,0,0,0,0,0,25119,525252,182528,2D+3D,Benzoic Acid,Agrochemical Information|Biological Test Resul...,17,155|157|161|165|167|175|421|426|427|433|434|43...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,D000890 - Anti-Infective Agents > D000935 - An...
1,1176,Urea,urea|carbamide|57-13-6|Carbonyldiamide|Isourea...,60.056,CH4N2O,69.1,29.0,-1.4,4,2,1,0,"InChI=1S/CH4N2O/c2-1(3)4/h(H4,2,3,4)",C(=O)(N)N,C(=O)(N)N,XSQUKJJJFZCRTK-UHFFFAOYSA-N,urea,60.0324,60.0324,0,1,0,0,0,0,0,0,0,187455,17279,7949,2D+3D,Urea,Agrochemical Information|Biological Test Resul...,17,200|330|421|426|427|433|434|435|445|530|540|54...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,B - Blood and blood forming organs > B05 - Blo...
2,3314,Eugenol,eugenol|97-53-0|4-Allyl-2-methoxyphenol|4-Ally...,164.200,C10H12O2,29.5,145.0,2.0,12,1,2,3,InChI=1S/C10H12O2/c1-3-4-8-5-6-9(11)10(7-8)12-...,COC1=C(C=CC(=C1)CC=C)O,COC1=C(C=CC(=C1)CC=C)O,RRAFCDWBNXTKKO-UHFFFAOYSA-N,2-methoxy-4-prop-2-enylphenol,164.0840,164.0840,0,1,0,0,0,0,0,0,0,10694,102401,32744,2D+3D,Eugenol,Agrochemical Information|Biological Test Resul...,17,155|157|161|165|167|175|248|330|421|426|427|43...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,C78272 - Agent Affecting Nervous System > C245...
3,1110,Succinic Acid,succinic acid|butanedioic acid|110-15-6|Amber ...,118.090,C4H6O4,74.6,92.6,-0.6,8,2,4,3,"InChI=1S/C4H6O4/c5-3(6)1-2-4(7)8/h1-2H2,(H,5,6...",C(CC(=O)O)C(=O)O,C(CC(=O)O)C(=O)O,KDYFGRWQOYBRFD-UHFFFAOYSA-N,butanedioic acid,118.0270,118.0270,0,1,0,0,0,0,0,0,0,22774,819,481,2D+3D,Succinic Acid,Agrochemical Information|Biological Test Resul...,17,155|157|161|165|167|175|192|194|248|256|270|32...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,COVID-19|COVID19|Coronavirus|Corona-virus|SARS...
4,4173,Metronidazole,metronidazole|443-48-1|Flagyl|Metronidazol|2-M...,171.150,C6H9N3O3,83.9,170.0,0.0,12,1,4,2,InChI=1S/C6H9N3O3/c1-5-7-4-6(9(11)12)8(5)2-3-1...,CC1=NC=C(N1CCO)[N+](=O)[O-],CC1=NC=C(N1CCO)[N+](=O)[O-],VAOCPAMSLUNLGC-UHFFFAOYSA-N,2-(2-methyl-5-nitroimidazol-1-yl)ethanol,171.0640,171.0640,0,1,0,0,0,0,0,0,0,39119,124678,34112,2D+3D,Metronidazole,Agrochemical Information|Biological Test Resul...,17,179|188|192|194|200|210|212|222|256|270|274|28...,20050325,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,A - Alimentary tract and metabolism > A01 - St...


In [12]:
print(df_compound.shape)

(55968, 40)


In [13]:
Compound_LeadingTo_Cancer = set()

In [14]:

for index, row in df_compound.iterrows():
     if row['cid'] not in Compound_LeadingTo_Cancer:
          db.create_node(int(row['cid']), "Compound", attribute1=row['cid'], attribute2=row['mw'])
          Compound_LeadingTo_Cancer.add(row['cid'])
     

In [15]:
df_bioassay_bio.head()

,aid,aidtype,aidname,aiddesc,aidcomment,aidsrcid,aidsrcname,aidextid,depdate,aidmdate,cids,sids,geneids,aidcategories,protacxns,depcatg,pmids,rnai,ecs,repacxns,taxids,cellids,targettaxid,cnt,activecnt,anatomyid,anatomy,annotation
0,1764210,Literature-derived,Cytotoxicity against patient-derived human col...,"Title: Synthesis of novel 1H-Pyrazolo[3,4-b]py...",Journal: Bioorg Med Chem Lett_||_Year: 2021_||...,43,ChEMBL,CHEMBL4823208,20220830,20220830,164625371,469821412,NaN,Toxicity,NaN,Curation Efforts|Research and Development,34182093,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,Colon cancer organoid,NaN
1,1259258,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_SECONDARY_SW48,20170330,20171011,NaN,56475149|56475150|56475471|56475472|56475473|5...,90|204|659|673|1021|1032|1108|1326|1459|2869|2...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,4182.0,NaN,160.0,40.0,NaN,NaN,NaN
2,1259259,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_PRIMARY_SW1463,20170330,20171011,NaN,56478465|56478467|56478468|56478474|56478482|5...,25|27|48|90|91|92|93|94|132|156|157|203|204|20...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,4176.0,NaN,794.0,9.0,NaN,NaN,NaN
3,1259260,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_PRIMARY_NCIH716,20170330,20171011,NaN,56478465|56478467|56478468|56478474|56478482|5...,25|27|48|90|91|92|93|94|132|156|157|203|204|20...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,4041.0,NaN,794.0,13.0,NaN,NaN,NaN
4,1259261,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_PRIMARY_HCC2998,20170330,20171011,NaN,56478465|56478467|56478468|56478474|56478482|5...,25|27|48|90|91|92|93|94|132|156|157|203|204|20...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,3729.0,NaN,794.0,10.0,NaN,NaN,NaN


In [16]:
print(get_cids_from_sid(196662829))

Failed to retrieve compound IDs for substance ID 196662829: 404
[False, -999]


In [17]:
#read substance csv 
file_path_substance = 'data/substance.csv'
df_substance = pd.read_csv(file_path_substance, sep=',')
#filter row that column cid not NAN 
df_substance_clean_cidNAN = df_substance[df_substance['cid'].notna()]


In [18]:
print(Compound_LeadingTo_Cancer)

{20971520, 5373954, 7995395, 39976969, 19267594, 4325386, 6553614, 46268432, 19, 46268435, 46268437, 46268440, 97386526, 3145764, 53084205, 110100532, 655413, 44826679, 20971581, 2883649, 3145794, 4849729, 3801158, 110100555, 85, 6422615, 4456535, 131171, 3145828, 2883685, 4325481, 106, 5242988, 4194415, 16646257, 262258, 39977074, 119, 655485, 2752638, 131204, 137, 138, 110100620, 13369486, 3408029, 2490525, 786602, 2228399, 176, 187, 44826812, 190, 199, 2359497, 204, 1310926, 1966297, 53608667, 222, 2097377, 2097378, 3408104, 7471340, 237, 238, 243, 248, 260, 110100742, 110100743, 110100748, 110100749, 273, 16253204, 110100763, 24248603, 3670313, 53608746, 17432876, 303, 304, 305, 44826930, 53608751, 110100783, 311, 313, 314, 13893946, 24248638, 20971841, 323, 20971844, 20971851, 3408207, 338, 4063572, 2097497, 20971874, 110100841, 16253293, 16384366, 110100845, 370, 3539317, 110100857, 2359677, 2359682, 655753, 41419165, 3801502, 46268831, 3932578, 110100902, 3408295, 424, 4325803, 

In [19]:
print(get_cids_from_sid(319885277))

Substance ID: 319885277, Compound IDs: [3390452]
[True, [3390452]]


In [20]:
#check type of column cid
print(df_substance_clean_cidNAN['cid'].dtypes)
#filter cid to int
df_substance_clean_cidNAN['cid'] = df_substance_clean_cidNAN['cid'].astype(int)
print(df_substance_clean_cidNAN['cid'].dtypes)

display(df_substance_clean_cidNAN[0:100])

float64
int64


/var/folders/tw/ksffzqq93t56xq1d5v8_ldyc0000gn/T/ipykernel_1921/3866176657.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_substance_clean_cidNAN['cid'] = df_substance_clean_cidNAN['cid'].astype(int)


,sid,cid,sidsrcname,sidextid,subssynonym,sidmdate,depdate,depcatg,annotation
3414,319885245,1001306,"Keith Lab, Institute of Cancer Science, Univer...",000A-0471,NaN,20161209,20161209,Research and Development,NaN
3415,319885261,794372,"Keith Lab, Institute of Cancer Science, Univer...",1718-0439,NaN,20161209,20161209,Research and Development,NaN
3416,319885277,3390452,"Keith Lab, Institute of Cancer Science, Univer...",3161-0502,NaN,20161209,20161209,Research and Development,NaN
3417,319885293,135454434,"Keith Lab, Institute of Cancer Science, Univer...",3643-1582,NaN,20190125,20161209,Research and Development,NaN
3418,319885309,846995,"Keith Lab, Institute of Cancer Science, Univer...",4314-0211,NaN,20190210,20161209,Research and Development,NaN
...,...,...,...,...,...,...,...,...,...
3509,319886765,20106910,"Keith Lab, Institute of Cancer Science, Univer...",ST011598,"(4S,6R)-6-hydroxy-6-methyl-5-(2-methylpyridyl)...",20210803,20161209,Research and Development,NaN
3510,319886781,2854374,"Keith Lab, Institute of Cancer Science, Univer...",ST011646,"N-[6-methyl-3-(morpholin-4-ylcarbonyl)(4,5,6,7...",20161209,20161209,Research and Development,NaN
3511,319886797,535784,"Keith Lab, Institute of Cancer Science, Univer...",ST011902,"1,3-dimethyl-1,3,4,6,3a,6a-hexahydro-1,3,4,6-t...",20210803,20161209,Research and Development,NaN
3512,319886813,304,"Keith Lab, Institute of Cancer Science, Univer...",ST012000,"14-(1,5-dimethylhexyl)-2,15-dimethyltetracyclo...",20161209,20161209,Research and Development,NaN


In [21]:
SubstanceToCompound = dict() 
for index, row in df_substance_clean_cidNAN.iterrows():
     if int(row['sid']) not in SubstanceToCompound:
          SubstanceToCompound[row['sid']] = int(row['cid'])
     else:
          print("error") 
print(SubstanceToCompound)

{319885245: 1001306, 319885261: 794372, 319885277: 3390452, 319885293: 135454434, 319885309: 846995, 319885325: 802038, 319885341: 1081554, 319885357: 16473313, 319885373: 930915, 319885389: 1204541, 319885405: 1902513, 319885421: 7045517, 319885437: 4179050, 319885453: 1079860, 319885469: 5059356, 319885485: 2950450, 319885501: 1152509, 319885517: 1094025, 319885533: 49878051, 319885549: 6106890, 319885565: 2201505, 319885581: 912656, 319885597: 4399749, 319885613: 942832, 319885629: 2957911, 319885645: 2944259, 319885661: 945648, 319885677: 49878323, 319885693: 135573166, 319885709: 15995772, 319885725: 3162395, 319885741: 22431314, 319885757: 3684746, 319885773: 6625124, 319885789: 20854746, 319885805: 20857413, 319885821: 16008119, 319885837: 20862278, 319885853: 20862966, 319885869: 5639799, 319885885: 4137758, 319885901: 6622418, 319885917: 5310484, 319885933: 20867720, 319885949: 20868369, 319885965: 20868934, 319885981: 16798683, 319885997: 20873589, 319886013: 18559167, 319886

In [22]:
print(SubstanceToCompound[319885245])
print(3390452 in Compound_LeadingTo_Cancer)

1001306
True


In [23]:
print(df_substance_clean_cidNAN.shape) 
print(len(Compound_LeadingTo_Cancer))

(55676, 9)
55968


KeyError: 469821412

In [26]:
#remove nan in sids column 
df_bioassay_cleanNaN = df_bioassay_bio.dropna(subset=['cids'])
df_bioassay_cleanNaN = df_bioassay_cleanNaN.dropna(subset=['sids'])

display(df_bioassay_cleanNaN)
print(df_bioassay_cleanNaN.shape) 
print(df_bioassay_bio.shape)


,aid,aidtype,aidname,aiddesc,aidcomment,aidsrcid,aidsrcname,aidextid,depdate,aidmdate,cids,sids,geneids,aidcategories,protacxns,depcatg,pmids,rnai,ecs,repacxns,taxids,cellids,targettaxid,cnt,activecnt,anatomyid,anatomy,annotation
0,1764210,Literature-derived,Cytotoxicity against patient-derived human col...,"Title: Synthesis of novel 1H-Pyrazolo[3,4-b]py...",Journal: Bioorg Med Chem Lett_||_Year: 2021_||...,43,ChEMBL,CHEMBL4823208,20220830,20220830,164625371,469821412,NaN,Toxicity,NaN,Curation Efforts|Research and Development,34182093,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,Colon cancer organoid,NaN
52,1733006,Confirmatory,Growth inhibition of human colon cancer cell l...,"Title: Discovery of 4-alkoxy-2-aryl-6,7-dimeth...",Compounds with activity <= 10uM or explicitly ...,43,ChEMBL,CHEMBL4738934,20220318,20220830,162646831|162650206|162664933,461519887|461524749|461545721,NaN,NaN,NaN,Curation Efforts|Research and Development,33631697,0,NaN,NaN,NaN,129058.0,NaN,3.0,2.0,NaN,Colon,NaN
53,1820036,Literature-derived,Anticancer activity against KOES-007 patient d...,"Title: Design, Synthesis, and Biological Evalu...",Journal: J Med Chem_||_Year: 2021_||_Volume: 6...,43,ChEMBL,CHEMBL5042739,20230314,20230314,166631882,475991183,NaN,NaN,NaN,Curation Efforts|Research and Development,33876637,0,NaN,NaN,9606,NaN,NaN,1.0,1.0,NaN,Stomach cancer organoid,NaN
54,1820038,Literature-derived,Anticancer activity against KOES-007 patient d...,"Title: Design, Synthesis, and Biological Evalu...",Journal: J Med Chem_||_Year: 2021_||_Volume: 6...,43,ChEMBL,CHEMBL5042741,20230314,20230314,166631882,475991183,NaN,NaN,NaN,Curation Efforts|Research and Development,33876637,0,NaN,NaN,9606,NaN,NaN,1.0,NaN,NaN,Stomach cancer organoid,NaN
55,1896227,Confirmatory,Antitumor activity against patient-derived Lun...,Title: Targeting Extracellular Signal-Regulate...,Compounds with activity <= 10uM or explicitly ...,43,ChEMBL,CHEMBL5152489,20230629,20230629,90331177,482060909,NaN,NaN,NaN,Curation Efforts|Research and Development,36205714,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337725,1918742,Literature-derived,Cytotoxicity against human PNT2 cells assessed...,"Title: Meroantarctines A-C, Meroterpenoids wit...",Journal: J Nat Prod_||_Year: 2022_||_Volume: 8...,43,ChEMBL,CHEMBL5218264,20230629,20230629,21674268|168298110|168298899|168299527,482088497|482089588|482089947|482090450,NaN,Toxicity,NaN,Curation Efforts|Research and Development,36413729.0,0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
337726,1918743,Literature-derived,Cytotoxicity against human RWPE-1 cells assess...,"Title: Meroantarctines A-C, Meroterpenoids wit...",Journal: J Nat Prod_||_Year: 2022_||_Volume: 8...,43,ChEMBL,CHEMBL5218265,20230629,20230629,21674268|168298110|168298899|168299527,482088497|482089588|482089947|482090450,NaN,Toxicity,NaN,Curation Efforts|Research and Development,36413729.0,0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
337727,1918744,Literature-derived,Cytotoxicity against docetaxel-resistant human...,"Title: Meroantarctines A-C, Meroterpenoids wit...",Journal: J Nat Prod_||_Year: 2022_||_Volume: 8...,43,ChEMBL,CHEMBL5218266,20230629,20230629,21674268|168298110|168298899|168299527,482088497|482089588|482089947|482090450,NaN,Toxicity,NaN,Curation Efforts|Research and Development,36413729.0,0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
337728,1918745,Literature-derived,Cytotoxicity against docetaxel-resistant human...,"Title: Meroantarctines A-C, Meroterpenoids wit...",Journal: J Nat Prod_||_Year: 2022_||_Volume: 8...,43,ChEMBL,CHEMBL5218267,20230629,20230629,148124,103172104,NaN,Toxicity,NaN,Curation Efforts|Research and Development,36413729.0,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN


(288951, 28)
(337730, 28)


In [ ]:
print(get_cids_from_sid(103172104))

Substance ID: 103172104, Compound IDs: [148124]
[True, [148124]]


In [ ]:
def GetResultJsonFromAid(aid:int):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{aid}/JSON"
    response = requests.get(url)
    if response.status_code == 200:
        assay_data = response.json()
        # print("______________________________ch____________________________")
        # print(json.dumps(assay_data['PC_AssaySubmit']['data'] , indent=4))
        return [True,assay_data['PC_AssaySubmit']['data'] ]
            
    else:
        print(f"Failed to retrieve assay results: {response.status_code}")
        return [False, -999]

In [ ]:
print(SubstanceToCompound[461519887])

KeyError: 461519887

In [ ]:
#4 -> Unspecified	
#3 -> Inactive
#2 -> Active
#1 -> Active agonist
#PUBCHEM AID =	1733006 sid is repeat in json respone
#for index, row in df_bioassay_cleanNaN.iterrows():
check = dict() 
i = 0
for index, row in df_bioassay_cleanNaN.iterrows():
    aid = row['aid'] 
    Jsons = GetResultJsonFromAid(aid)[1]
    #for Jsons 
    db.create_node(int(aid), "Bioassay", attribute1=row['aidname'], attribute2=row['aiddesc'])
    for Json in Jsons:
        # print("1",Json)
        # print("2",Json['sid'])
        # print("3",Json['outcome'])
        if isinstance(Json['outcome'], int):
            print(69696969)
            if Json['sid'] not in SubstanceToCompound:
                continue
            print(type(Json['outcome']))
            print(Json['outcome']) 
            if Json['outcome'] == 2:
                print("yes2")
                #create relation between compound and bioassay in neo4j
                db.create_relationship(int(SubstanceToCompound[Json['sid']]), "ActiveOn", int(aid))
            elif Json['outcome'] == 3: 
                print("yes3")
                db.create_relationship(int(SubstanceToCompound[Json['sid']]), "InActiveOn",  int(aid))
    i+=1 
    if i == 20:
        break
print(check) 

    


69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
69696969
6

In [29]:
# Create two nodes with different labels and attributes
db.create_node("Node1", "Label1", attribute1="value1")
db.create_node("Node2", "Label2", attribute2="value2")

# Create a relationship between the two nodes with attributes
db.create_relationship("Node1", "RELATIONSHIP_TYPE", "Node2")


In [ ]:
print(SubstanceToCompound[319885245] in Compound_LeadingTo_Cancer)

True


In [ ]:
# note request form aj 1. 